using langchain's built in classes to support retrieving document parents instead of chunks

-   bigger embedding model cuz why not
-   recursive text splitting


In [1]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore  # ...
from langchain_redis import RedisConfig, RedisVectorStore
from langchain_community.storage import RedisStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from bs4 import BeautifulSoup
from transformers import AutoTokenizer
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
import re
import redis

c:\Users\Andrew\miniforge3\lib\site-packages\langchain\retrievers\document_compressors\chain_extract.py:15: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain.chains.llm import LLMChain
c:\Users\Andrew\miniforge3\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


In [2]:
def get_html(file, sep=" "):

    # stolen from https://stackoverflow.com/a/75501596 this sucks
    NON_BREAKING_ELEMENTS = [
        "a",
        "abbr",
        "acronym",
        "audio",
        "b",
        "bdi",
        "bdo",
        "big",
        "button",
        "canvas",
        "cite",
        "code",
        "data",
        "datalist",
        "del",
        "dfn",
        "em",
        "embed",
        "i",
        "iframe",
        "img",
        "input",
        "ins",
        "kbd",
        "label",
        "map",
        "mark",
        "meter",
        "noscript",
        "object",
        "output",
        "picture",
        "progress",
        "q",
        "ruby",
        "s",
        "samp",
        "script",
        "select",
        "slot",
        "small",
        "span",
        "strong",
        "sub",
        "sup",
        "svg",
        "template",
        "textarea",
        "time",
        "u",
        "tt",
        "var",
        "video",
        "wbr",
    ]

    def html_to_text(text, preserve_new_lines=True, strip_tags=["style", "script"]):
        soup = BeautifulSoup(text, "html.parser")
        for element in soup(strip_tags):
            element.extract()
        prev = False
        if preserve_new_lines:
            for element in soup.find_all():
                strings = element.find_all(string=True, recursive=False)
                if strings is not None:
                    strings = [
                        s
                        for s in (None if s.strip() == "" else s for s in strings)
                        if s is not None
                    ]
                strings = strings is not None and strings != []
                strings = True
                if element.name not in NON_BREAKING_ELEMENTS and strings:
                    (
                        element.append("\n")
                        if element.name == "br"
                        else element.append("\n\n")
                    )
        return soup.get_text(separator="")  # close enough

    def replace_newlines(text):
        text = re.sub(r"\n{3}", "\n", text)
        text = re.sub(r"\n{4,}", "\n\n", text)
        return text

    with open(file, "r") as f:
        soup = BeautifulSoup(f, "html.parser")
        iwant = soup.find_all("div", {"id": "content"})
        assert len(iwant) == 1
        text = html_to_text(str(iwant[0]))
        # text = soup.get_text(separator=sep)
        # text = str(iwant[0]) # recursivecharactertextsplitter has html splitting built in so i will try that
        # it sucks nevermind
        text = replace_newlines(text).strip()
        return text  # idk how to do better idk why there's just double spaces sometimes
    return "?"

In [3]:
dataset_path = "../dataset/raw_html"
files = os.listdir(dataset_path)
files

['Accounts.html',
 'Accounts_Collaborator.html',
 'ActiveDirectory.html',
 'AddingUMIACSCertificateAuthority.html',
 'Adobe.html',
 'AlternativePickup.html',
 'Apptainer.html',
 'Archives.html',
 'ATL_ConferenceRooms.html',
 'Automounter.html',
 'Backups.html',
 'BarracudaSpamFirewall.html',
 'BarracudaSpamFirewall_QuarantinePassthrough.html',
 'BarracudaSpamFirewall_Scoring.html',
 'BarracudaSpamFirewall_SearchingQuarantine.html',
 'Bash.html',
 'BashForWindows.html',
 'BitLocker.html',
 'BitLocker_PersonalUse.html',
 'Branding.html',
 'Caffe.html',
 'CBCB.html',
 'CCompilers.html',
 'CDebuggers.html',
 'CIFS.html',
 'ClassAccounts.html',
 'ClassAccounts_Manage.html',
 'Clear_Cache.html',
 'CloudDataStorage.html',
 'CMake.html',
 'CML.html',
 'CompromisedPasswordFiltering.html',
 'ComputationalResource.html',
 'Compute_DataLocality.html',
 'ConferenceRooms.html',
 'ConferenceRooms_Recording.html',
 'ConferenceRooms_Solstice.html',
 'ConferenceRooms_TouchPanel.html',
 'ConferenceRooms_

In [4]:
print(get_html("../dataset/raw_html/Nexus.html"))

Nexus

From UMIACS

Jump to navigation
Jump to search
The Nexus is the combined scheduler of resources in UMIACS.  The resource manager for Nexus is SLURM.  Resources are arranged into partitions where users are able to schedule computational jobs.  Users are arranged into a number of SLURM accounts based on faculty, lab, or center investments.

Contents

1 Getting Started

1.1 Access
1.2 Jobs

1.2.1 Interactive
1.2.2 Batch

2 Partitions
3 Quality of Service (QoS)

3.1 Job QoS
3.2 Partition QoS


4 Storage

4.1 Home Directories
4.2 Scratch Directories

4.2.1 Network Scratch Directories
4.2.2 Local Scratch Directories


4.3 Faculty Allocations
4.4 Project Allocations
4.5 Datasets

Getting Started
All accounts in UMIACS are sponsored.  If you don't already have a UMIACS account, please see Accounts for information on getting one.  You need a full UMIACS account (not a  collaborator account) in order to access Nexus.

Access
Your access to submission nodes (alternatively called login node

In [5]:
pages = []

for f in files:
    pages.append(get_html(dataset_path + "/" + f))
pages

['Accounts\n\nFrom UMIACS\n\nJump to navigation\nJump to search\nThis page outlines the various account types at UMIACS and their management methods.\nDo not share any UMIACS-affiliated account for any purpose. Action may be taken against shared accounts.\n\nContents\n\n1 UMIACS Account\n2 Guest / Collaborator Account\n3 Request a UMIACS account\n4 Password Requirements\n5 Changing account password\n\n5.1 If you know your current password\n5.2 If you do not know your current password\n\n\n6 Sponsored Accounts\n7 All Accounts Related Pages\n\n\nUMIACS Account\nA full UMIACS account consists of the following components:\n\n Active Directory Account\nProvides access to UMIACS-supported computers,  secgroup controlled resources, web services such as GitLab, and the VPN.\n Jira Account\nProvides access to the JIRA Request Tracker. Please note that it is possible to set a separate password for this account, at which point it will no longer be synced from your Active Directory Account. Changi

In [6]:
docs = []
for i, page in enumerate(pages):
    docs.append(Document(page, metadata={"name": files[i]}))
docs

[Document(metadata={'name': 'Accounts.html'}, page_content='Accounts\n\nFrom UMIACS\n\nJump to navigation\nJump to search\nThis page outlines the various account types at UMIACS and their management methods.\nDo not share any UMIACS-affiliated account for any purpose. Action may be taken against shared accounts.\n\nContents\n\n1 UMIACS Account\n2 Guest / Collaborator Account\n3 Request a UMIACS account\n4 Password Requirements\n5 Changing account password\n\n5.1 If you know your current password\n5.2 If you do not know your current password\n\n\n6 Sponsored Accounts\n7 All Accounts Related Pages\n\n\nUMIACS Account\nA full UMIACS account consists of the following components:\n\n Active Directory Account\nProvides access to UMIACS-supported computers,  secgroup controlled resources, web services such as GitLab, and the VPN.\n Jira Account\nProvides access to the JIRA Request Tracker. Please note that it is possible to set a separate password for this account, at which point it will no l

In [7]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)  # characters not toks

idk what makes good chunks for when we're retrieving parent anyways. the number is characters and i tried 500 characters and sometimes it'd split out just single lines (which seems max useless because it'll basically never be a good match compared to longer chunks)


In [8]:
out = child_splitter.split_documents([docs[212]])
print(docs[0].page_content)
for o in out:
    print(o.page_content)
    print("SPLIT-------------------------")

Accounts

From UMIACS

Jump to navigation
Jump to search
This page outlines the various account types at UMIACS and their management methods.
Do not share any UMIACS-affiliated account for any purpose. Action may be taken against shared accounts.

Contents

1 UMIACS Account
2 Guest / Collaborator Account
3 Request a UMIACS account
4 Password Requirements
5 Changing account password

5.1 If you know your current password
5.2 If you do not know your current password


6 Sponsored Accounts
7 All Accounts Related Pages


UMIACS Account
A full UMIACS account consists of the following components:

 Active Directory Account
Provides access to UMIACS-supported computers,  secgroup controlled resources, web services such as GitLab, and the VPN.
 Jira Account
Provides access to the JIRA Request Tracker. Please note that it is possible to set a separate password for this account, at which point it will no longer be synced from your Active Directory Account. Changing your Jira password DOES NOT ch

would use redis for doc store but langchain only has vector and byte store classes for redis and im not writing my own class (because ParentDocumentRetriever requires a subclass of an abstract class to work)


In [9]:
store = InMemoryStore()

In [10]:
# 1.5B version because why not.
embedding_model = HuggingFaceEmbeddings(
    model_name="dunzhang/stella_en_1.5B_v5",
    model_kwargs={
        "trust_remote_code": True,
    },
)

In [26]:
config = RedisConfig(
    index_name="umiacs",
    redis_url="redis://localhost:6379",
    metadata_schema=[
        {"name": "doc_id", "type": "tag"},
        {"name": "name", "type": "tag"},
    ],
)  # i have to manually add metadata field for this to work XD
redis_conn = redis.from_url("redis://localhost:6379", db=0)
vectorstore = RedisVectorStore(embedding_model, config)

In [27]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    search_kwargs={},  # one Billion bugs.
)

In [28]:
retriever.add_documents(docs)  # why does 1.5 B model not fit in 12 GB ?
# also it's 1018 chunks

In [29]:
def delete_all_vectors(redis_conn, index_name):
    """Deletes all keys in the given Redis namespace."""
    try:
        redis_conn.flushdb()
        print(f"All vectors from index '{index_name}' have been deleted.")
    except Exception as e:
        print(f"Error deleting vectors: {e}")

In [36]:
len(redis_conn.keys())

1010

In [37]:
len(list(store.yield_keys()))

264

In [31]:
sub_docs = vectorstore.similarity_search_with_score("python notebook nexus")
print(sub_docs[0])

(Document(metadata={'doc_id': '8030a557-3f07-4d6e-b256-5a75e1673e8e', 'name': 'SLURM.html'}, page_content='jupyter notebook --no-browser --port=8889 --ip=0.0.0.0\n\nThis will start running the notebook on port 8889. Note: You must keep this shell window open to be able to connect. If the submission node for the cluster you are using is not accessible via the public internet, you must also be on a machine connected to the UMIACS network or connected to our  VPN in order to access the Jupyter notebook once you start the SSH tunnel, so ensure this is the case before starting the tunnel. Then, on your local machine, run\n\nssh -N -f -L localhost:8888:<NODENAME>:8889 <USERNAME>@<SUBMISSIONNODE>.umiacs.umd.edu'), 0.41864824295)


In [32]:
retrieved_docs = retriever.invoke("python notebook nexus")
for d in retrieved_docs:
    print(d)
print(len(retrieved_docs))

dict_values([TextField(name='text', type='text', path=None, attrs=TextFieldAttributes(sortable=False, weight=1, no_stem=False, withsuffixtrie=False, phonetic_matcher=None)), FlatVectorField(name='embedding', type='vector', path=None, attrs=FlatVectorFieldAttributes(dims=1024, algorithm=<VectorIndexAlgorithm.FLAT: 'FLAT'>, datatype=<VectorDataType.FLOAT32: 'FLOAT32'>, distance_metric=<VectorDistanceMetric.COSINE: 'COSINE'>, initial_cap=None, block_size=None)), TagField(name='doc_id', type='tag', path=None, attrs=TagFieldAttributes(sortable=False, separator='|', case_sensitive=False, withsuffixtrie=False)), TagField(name='name', type='tag', path=None, attrs=TagFieldAttributes(sortable=False, separator='|', case_sensitive=False, withsuffixtrie=False))])
page_content='SLURM

From UMIACS

Jump to navigation
Jump to search
Contents

1 Slurm Workload Manager

1.1 Documentation
1.2 Commands

1.2.1 srun
1.2.2 salloc
1.2.3 sbatch
1.2.4 squeue
1.2.5 scancel
1.2.6 sacct
1.2.7 sstat


1.3 Modules
1

find dump.rdb and save it or something


In [33]:
import pickle

with open("parentdoc.pkl", "wb") as out:
    pickle.dump(store, out, -1)

In [34]:
%load_ext autoreload
%autoreload 2

In [35]:
retriever._new_arg_supported

True

conclusion

1. langchain-redis is more "official" but it omega sucks and some things seem completely broken
2. i cant figure out how to make parentdocumentretriever actually retrieve things the way i want to
    1. no matter where i put {"k": 0} in the arguments it doesn't do it so idk. also just looking at the code i cant find anything that accepts those arguments for some reason
    2. so i will just manually call the search and document references which isnt bad i guess
3. redis local for some reason has different module for vector searches ...
4. also i need to get rid of allpages from the list of pages cuz it messes up a bunch of queries
